In [0]:
%pip install yfinance
%restart_python

In [0]:
import yfinance as yf
from datetime import datetime, date
import os
import time
from pyspark.sql.functions import lit
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException
from pyspark.sql import functions as F
from pyspark.sql.types import DateType, TimestampType

# Definicja sesji Spark (dla środowisk poza Databricks)
# Spark w Databricks jest dostępny automatycznie.
# spark = SparkSession.builder.appName("YfinanceDownloader").getOrCreate()

# --- KROK 1: Tworzenie df_tickets z tabeli Databricks SQL ---
try:
    sql_query = "SELECT Ticket, company_name FROM google_drive.gpw_data"
    spark_df_tickets = spark.sql(sql_query)
    df_tickets_list = spark_df_tickets.collect()
    company_info_dict = {row['Ticket']: row['company_name'] for row in df_tickets_list}
    gpw_ticket_list = list(company_info_dict.keys())
    print("Pomyślnie utworzono Spark DataFrame z tabeli SQL.")
    print("Akcje do przetworzenia:", gpw_ticket_list)
except Exception as e:
    print(f"Błąd odczytu z tabeli SQL: {e}.")
    gpw_ticket_list = []
    company_info_dict = {}


In [0]:
# Ustawienia katalogu Unity Catalog Volume
catalog_name = "workspace"
schema_name = "default"
volume_name = "yfinance_data"
base_output_directory = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/yfinance_parquets/"

# Utworzenie katalogu bazowego, jeśli nie istnieje
try:
    dbutils.fs.mkdirs(base_output_directory)
    print(f"Pomyślnie utworzono katalog: {base_output_directory}")
except Exception as e:
    print(f"Katalog już istnieje lub wystąpił błąd podczas jego tworzenia: {e}")

# Ustawienia zakresu dat dla yfinance
end_date = date.today()

if not gpw_ticket_list:
    print("Brak danych firm do przetworzenia. Używanie danych zastępczych dla demonstracji.")




In [0]:
from pyspark.sql.functions import to_date
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql import types as T

print("\nRozpoczynanie pobierania danych z yfinance i zapisywanie do pojedynczych plików Parquet...")

# Helper function to check if a path exists using dbutils.fs.ls
def path_exists(path):
    try:
        dbutils.fs.ls(path)
        return True
    except Exception:
        return False
    
# Lista kolumn, które mają być ujednolicone
price_columns = ["Close", "High", "Low", "Open"]
# Docelowa precyzja: 10 cyfr łącznie, 4 po przecinku (Decimal(10, 4))
target_decimal_type = T.DecimalType(10, 4) 

for t in gpw_ticket_list:
    yfinance_ticker = f"{t}.WA"
    output_path = os.path.join(base_output_directory, f"ticker={t}")
    print(f"Przetwarzanie akcji: {yfinance_ticker}")

    # Sprawdzenie, czy katalog dla danego tickera już istnieje
    if path_exists(output_path):
        try:
            # Plik istnieje, próbujemy odczytać i zaktualizować
            existing_df = spark.read.parquet(output_path)
            # Konwersja kolumny na typ DateType
            existing_df = existing_df.withColumn("Date", col("Date").cast("date"))
            last_download_date = existing_df.selectExpr("max(Date)").collect()[0][0]
            if last_download_date:
                today_date_obj = date.today()
                if last_download_date < today_date_obj:
                    print(f"Dane dla '{yfinance_ticker}' wymagają aktualizacji. Ostatnia data: {last_download_date}.")
                    start_date_for_update = (pd.to_datetime(last_download_date) + pd.Timedelta(days=1)).strftime('%Y-%m-%d')
                    data = yf.download(yfinance_ticker, start=start_date_for_update, end=end_date)
                    
                    if not data.empty:
                        df_new_data = data.reset_index()
                        if isinstance(df_new_data.columns, pd.MultiIndex):
                            df_new_data.columns = df_new_data.columns.map(lambda x: x[0])
                        df_new_data.info()
                        spark_new_data_df = spark.createDataFrame(df_new_data)
                        spark_new_data_df = spark_new_data_df.withColumn("Date", col("Date").cast("date"))
                        company_name = company_info_dict.get(t, 'N/A')
                        spark_new_data_df = spark_new_data_df.withColumn("Ticket", lit(t)).withColumn("Company_Name", lit(company_name))
                        
                        combined_spark_df = existing_df.unionByName(spark_new_data_df, allowMissingColumns=True).dropDuplicates(['Date'])
                        combined_spark_df.write.mode("overwrite").parquet(output_path)
                        print(f"Zaktualizowano dane dla '{yfinance_ticker}'. Dodano {spark_new_data_df.count()} nowe wiersze.")
                    else:
                        print(f"Brak nowych danych do pobrania dla '{yfinance_ticker}'.")
                else:
                    print(f"Dane dla '{yfinance_ticker}' są aktualne na dzień {last_download_date}. Pomijanie.")
            else:
                print(f"Ostrzeżenie: Kolumna 'Date' nie znaleziona lub pusta dla '{yfinance_ticker}'. Pełne ponowne pobranie.")
                # Jeśli kolumna Date jest pusta, traktujemy to jak brak danych
                data = yf.download(yfinance_ticker, period='max', end=end_date)
                
                if not data.empty:
                    df_new_data = data.reset_index()
                    if isinstance(df_new_data.columns, pd.MultiIndex):
                        df_new_data.columns = df_new_data.columns.map(lambda x: x[0])
                    spark_new_data_df = spark.createDataFrame(df_new_data)
                    spark_new_data_df = spark_new_data_df.withColumn("Date", col("Date").cast("date"))
                    company_name = company_info_dict.get(t, 'N/A')
                    spark_new_data_df = spark_new_data_df.withColumn("Ticket", lit(t)).withColumn("Company_Name", lit(company_name))
                    for col_name in price_columns:
                        spark_new_data_df = spark_new_data_df.withColumn(col_name,
                        F.round(F.col(col_name), 4).cast(target_decimal_type))
                    spark_new_data_df.write.mode("overwrite").parquet(output_path)
                    print(f"Pobrano i zapisano pełne dane dla '{yfinance_ticker}'.")
                else:
                    print(f"Brak danych dla '{yfinance_ticker}'.")

        except Exception as e:
            print(f"Wystąpił błąd podczas aktualizacji dla '{yfinance_ticker}': {e}. Przystąpienie do pełnego pobierania.")
            
    else:
        # Plik nie istnieje, zaczynamy od pełnego pobierania
        print(f"Brak istniejących danych dla '{yfinance_ticker}'. Rozpoczęcie pełnego pobierania.")
        try:
            data = yf.download(yfinance_ticker, period='max', end=end_date)
            print(data)
            if not data.empty:
                df_new_data = data.reset_index()
                if isinstance(df_new_data.columns, pd.MultiIndex):
                    df_new_data.columns = df_new_data.columns.map(lambda x: x[0])
                print(df_new_data)
               #df_new_data['Date'] = df_new_data['Date'].dt.strftime('%Y-%m-%d')
                spark_new_data_df = spark.createDataFrame(df_new_data)
                spark_new_data_df = spark_new_data_df.withColumn("Date", col("Date").cast("date"))
                company_name = company_info_dict.get(t, 'N/A')
                spark_new_data_df = spark_new_data_df.withColumn("Ticket", lit(t)).withColumn("Company_Name", lit(company_name))
                for col_name in price_columns:
                    spark_new_data_df = spark_new_data_df.withColumn(col_name,
                    F.round(F.col(col_name), 4).cast(target_decimal_type))
                spark_new_data_df.write.mode("overwrite").parquet(output_path)
                print(f"Pobrano i zapisano pełne dane dla '{yfinance_ticker}'.")
            else:
                print(f"Brak danych dla '{yfinance_ticker}'.")
        except Exception as e:
            print(f"Wystąpił błąd podczas pobierania lub zapisywania dla '{yfinance_ticker}': {e}")
            
    time.sleep(1) # Unikaj przeciążania API